# Importing Eye data


- [ ] get_pupildata() or the like
- [ ] re-implement PupilData to use EyeDataDict (use "switch_eye" function?)
- [x] plot_scanpath()
- [x] plot_heatmap()
- [ ] interpolation and preprocessing (transfer from pupil to eyetracking)

In [1]:
import sys, os
sys.path.insert(0,"..") # this is not needed if you have installed pypillometry
import pypillometry as pp
print(pp.__package_path__) # check that correct version is being used
import pandas as pd
import numpy as np
import pylab as plt

/opt/anaconda3/envs/pypil/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/mmi041/Dropbox/work/projects/pupil/pypillometry/pypillometry


In [2]:
## loading the raw samples from the asc file
fname_samples="../data/002_rlmw_samples.asc"
df=pd.read_table(fname_samples, index_col=False,
                  names=["time", "left_x", "left_y", "left_p",
                         "right_x", "right_y", "right_p"])

## Eyelink tracker puts "   ." when no data is available for x/y coordinates
left_x=df.left_x.values
left_x[left_x=="   ."] = np.nan
left_x = left_x.astype(float)

left_y=df.left_y.values
left_y[left_y=="   ."] = np.nan
left_y = left_y.astype(float)

right_x=df.right_x.values
right_x[right_x=="   ."] = np.nan
right_x = right_x.astype(float)

right_y=df.right_y.values
right_y[right_y=="   ."] = np.nan
right_y = right_y.astype(float)

/tmp/ipykernel_6074/799129609.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df=pd.read_table(fname_samples, index_col=False,


In [3]:
## Loading the events from the events file
fname_events="../data/002_rlmw_events.asc"
# read the whole file into variable `events` (list with one entry per line)
with open(fname_events) as f:
    events=f.readlines()

# keep only lines starting with "MSG"
events=[ev for ev in events if ev.startswith("MSG")]
experiment_start_index=np.where(["experiment_start" in ev for ev in events])[0][0]
events=events[experiment_start_index+1:]
df_ev=pd.DataFrame([ev.split() for ev in events])
df_ev=df_ev[[1,2]]
df_ev.columns=["time", "event"]
df_ev

,time,event
0,7269756,F_1
1,7270256,S_1_1_5
2,7271352,C_1_5_2
3,7271589,R_1_15
4,7273355,F_2
...,...,...
1315,8453467,R_300_18
1316,8455583,F_300
1317,8456083,P_300
1318,8456608,CP_300_1


In [30]:
from pypillometry import EyeData, EyeDataDict, keephistory, _inplace
from collections.abc import Iterable
from matplotlib import cm
import matplotlib.patches as patches
from pypillometry import PupilData


pp.EyeData.correct_pupil_foreshortening=correct_pupil_foreshortening

In [41]:
# Creating EyeData object that contains both X-Y coordinates
# and pupil data
d = pp.EyeData(time=df.time, name="002_ceylin",
               left_x=left_x, left_y=left_y, left_pupil=df.left_p,
               right_x=right_x, right_y=right_y, right_pupil=df.right_p,
               event_onsets=df_ev.time, event_labels=df_ev.event,
               keep_orig=True)\
            .reset_time()
d

EyeData(002_ceylin_va, 65.0MiB):
 n                   : 597301
 sampling_rate       : 500.0
 data                : ['left_x', 'left_y', 'left_pupil', 'right_x', 'right_y', 'right_pupil']
 nevents             : 1320
 screen_limits       : not set
 physical_screen_size: not set
 screen_eye_distance : not set
 nmiss               : 0
 perc_miss           : 0.0
 duration_minutes    : 19.910033333333335
 start_min           : 0.0
 end_min             : 19.91
 glimpse             : EyeDataDict(vars=6,n=597301): 
  left_x (float64): 655.6, 655.7, 655.0, 654.5, 655.0...
  left_y (float64): 599.9, 598.9, 597.6, 597.8, 597.8...
  left_pupil (float64): 1121.0, 1122.0, 1124.0, 1126.0, 1126.0...
  right_x (float64): 773.1, 773.8, 774.8, 776.3, 776.7...
  right_y (float64): 594.2, 593.9, 596.5, 597.4, 597.3...
  right_pupil (float64): 949.0, 951.0, 956.0, 959.0, 960.0...

 History:
 *
 └ fill_time_discontinuities()
  └ reset_time()

In [ ]:
%matplotlib 
# clean pupil data using XY saccade data
d.set_experimental_setup(
    screen_resolution=(1280, 1024), 
    physical_screen_size=("30 cm", "20 cm"),
    eye_to_screen_perpendicular="60 cm"
)
pd1=d.get_pupildata(eye="right")
d2 = d.correct_pupil_foreshortening(eyes="right")
pd2=d2.get_pupildata(eye="right")

plt.figure(figsize=(10,5))
pd1.plot( (20,30), units="sec")
pd2.plot( (20,30), units="sec")

Using matplotlib backend: TkAgg


In [50]:
np.any(pd1.sy==0)

True

In [46]:
pd2.blinks_detect(min_duration=pars["min_duration"],strategies=pars["strategies"],
                     vel_onset=pars["vel_onset"], vel_offset=pars["vel_offset"],
                     min_onset_len=pars["min_onset_len"], min_offset_len=pars["min_offset_len"])

IndexError: index -1 is out of bounds for axis 0 with size 0

In [43]:
#pd = d.get_pupildata("left")
pd=pd2
default_param={"min_duration":10,    # min duration of a blink
               "min_offset_len":2,   # offset/onset-length for blink-detection
               "min_onset_len":3,
               "vel_onset":-5,       # velocity thresholds for onset and offset
               "vel_offset":5,
               "strategies":["zero","velocity"],  # strategies for blink-detection
               "distance":100,        # minimum distance between two blinks
               "margin":(50,150),     # margins for Mahot algorithm
               "cutoff":5}            # lowpass-filter cutoff (Hz)
pars=default_param
pd3=pd.blinks_detect(min_duration=pars["min_duration"],strategies=pars["strategies"],
                     vel_onset=pars["vel_onset"], vel_offset=pars["vel_offset"],
                     min_onset_len=pars["min_onset_len"], min_offset_len=pars["min_offset_len"])\
            .blinks_merge_close(distance=pars["distance"])\
            .blinks_interp_mahot(margin=pars["margin"], vel_onset=pars["vel_onset"], vel_offset=pars["vel_offset"])\
            .lowpass_filter(cutoff=pars["cutoff"])
pd.plot_segments(overlay=pd2, pdffile="%s.pdf"%pd.name, ylim=(pd2.sy.min(), pd2.sy.max()))

IndexError: index -1 is out of bounds for axis 0 with size 0

In [8]:
def fixcross(key, trial=None):
    ttype,trialnum,*rest = key.split("_")
    if trial is not None:
        return ttype=="F" and trialnum==str(trial)
    else:
        return ttype=="F"
def response(key, trial=None):
    ttype,trialnum,*rest = key.split("_")
    if trial is not None:
        return (ttype=="R" or ttype=="RP") and trialnum==str(trial)
    else:
        return (ttype=="R" or ttype=="RP")
def trialno(key, trialno=None):
    return key.split("_")[1]==str(trialno)

d.test_selector(fixcross, trial=10)


array(['F_10'], dtype=object)

In [9]:

d.get_intervals("F_1", padding=(-200,200), units="ms")
ints = d.get_intervals(fixcross, padding=(-200,200), units="ms")
ints = d.get_intervals(trialno, padding=(-200,200), units="ms", trialno=10)
ints = d.get_intervals( (fixcross, response), padding=(-200,200), units="ms")
ints

d.plot_scanpath( ints[20], units="ms")
d.plot( ints[20], units="ms", plot_onsets="both")

In [10]:

d.plot_scanpath((0,10), units="sec", title="blub")

In [11]:
d.plot_heatmap(plot_eyes="left")

In [12]:
d.plot((3,14), units="sec", plot_onsets="both")